In [1]:
from process_bigraph import Composite

from biosimulators_processes import CORE
from biosimulators_processes.steps.bio_compose import MongoDatabaseEmitter
from process_bigraph.composite import RAMEmitter
from biosimulators_processes.processes.copasi_process import CopasiProcess

Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.

PLEASE NOTE: Smoldyn is not correctly installed on your system which prevents you from using the SmoldynProcess. Please refer to the README for further information on installing Smoldyn.


In [2]:
uri = 'mongodb://localhost:27017/?retryWrites=true&w=majority&appName=biosimulators-processes'
emitter = MongoDatabaseEmitter(config={'connection_uri': uri}, core=CORE)
ram = RAMEmitter(config={}, core=CORE)
ram.inputs()

{}

In [2]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

In [3]:
doc = {
    'copasi': {
        '_type': 'process',
        'address': 'local:copasi-process',
        'config': {
            'model': {
                'model_source': model_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'model_parameters': ['model_parameters_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:database-emitter',
        'config': {
            'emit': {
                'time': 'float',
                'floating_species_concentrations': 'tree[float]'
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store']
        }
    }
}


composite = Composite(config={'state': doc}, core=CORE)

composite.run(10)

found a filepath


/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/biosimulators_processes/processes/copasi_process.py:328: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [4]:
composite.gather_results()

{('emitter',): [{'time': 0.0,
   'floating_species_concentrations': {'LacI protein': 0.0,
    'TetR protein': 0.0,
    'cI protein': 0.0,
    'LacI mRNA': 0.0,
    'TetR mRNA': 20.0,
    'cI mRNA': 0.0},
   '_id': ObjectId('66df50b39694d784cca4dfb2')},
  {'time': 1.0,
   'floating_species_concentrations': {'LacI protein': 81.44046884923394,
    'TetR protein': 188.3821831030562,
    'cI protein': 42.64131125320374,
    'LacI mRNA': 19.90344166947679,
    'TetR mRNA': 50.61558010446934,
    'cI mRNA': 7.491001903157758},
   '_id': ObjectId('66df50b39694d784cca4dfb3')},
  {'time': 2.0,
   'floating_species_concentrations': {'LacI protein': 299.9792524293013,
    'TetR protein': 546.4092678039393,
    'cI protein': 127.23014070279017,
    'LacI mRNA': 41.13697667160224,
    'TetR mRNA': 74.22450737953378,
    'cI mRNA': 13.357268321541767},
   '_id': ObjectId('66df50b39694d784cca4dfb4')},
  {'time': 3.0,
   'floating_species_concentrations': {'LacI protein': 637.6017815600158,
    'TetR p

In [47]:
from abc import ABC, abstractmethod
from enum import Enum
from typing import *
from datetime import datetime

from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.database import Database


class DatabaseConnector(ABC):
    """Abstract class that is both serializable and interacts with the database (of any type). """
    def __init__(self, connection_uri: str, database_id: str, connector_id: str):
        self.database_id = database_id
        self.client = self._get_client(connection_uri)
        self.db = self._get_database(self.database_id)

    @staticmethod
    def timestamp() -> str:
        return str(datetime.utcnow())

    def refresh_data(self):
        def refresh_collection(coll):
            for job in self.db[coll].find():
                self.db[coll].delete_one(job)

        for collname in self.db.list_collection_names():
            refresh_collection(collname)

    @abstractmethod
    def _get_client(self, *args):
        pass

    @abstractmethod
    def _get_database(self, db_id: str):
        pass

    @abstractmethod
    async def read(self, *args, **kwargs):
        pass

    @abstractmethod
    async def write(self, *args, **kwargs):
        pass

    @abstractmethod
    def get_collection(self, **kwargs):
        pass


class JobStatus(Enum):
    PENDING = "PENDING"
    IN_PROGRESS = "IN_PROGRESS"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"


class DatabaseCollections(Enum):
    PENDING_JOBS = "PENDING_JOBS".lower()
    IN_PROGRESS_JOBS = "IN_PROGRESS_JOBS".lower()
    COMPLETED_JOBS = "COMPLETED_JOBS".lower()


class MultipleConnectorError(Exception):
    def __init__(self, message: str):
        self.message = message


class MongoDbConnector(DatabaseConnector):
    def __init__(self, connection_uri: str, database_id: str, connector_id: str = None):
        super().__init__(connection_uri, database_id, connector_id)

    def _get_client(self, *args):
        return MongoClient(args[0])

    def _get_database(self, db_id: str) -> Database:
        return self.client.get_database(db_id)

    def _get_jobs_from_collection(self, coll_name: str):
        return [job for job in self.db[coll_name].find()]
    
    @property
    def data(self):
        return self._get_data()
    
    def _get_data(self):
        return {coll_name: [v for v in self.db[coll_name].find()] for coll_name in self.db.list_collection_names()}
    
    def read(self, collection_name: DatabaseCollections | str, **kwargs):
        """Args:
            collection_name: str
            kwargs: (as in mongodb query)
        """
        coll_name = self._parse_enum_input(collection_name)
        coll = self.get_collection(coll_name)
        result = coll.find_one(kwargs.copy())
        return result

    def write(self, collection_name: DatabaseCollections | str, **kwargs):
        """
            Args:
                collection_name: str: collection name in mongodb
                **kwargs: mongo db `insert_one` query defining the document where the key is as in the key of the document.
        """
        coll_name = collection_name

        coll = self.get_collection(coll_name)
        result = coll.insert_one(kwargs.copy())
        return kwargs

    def get_collection(self, collection_name: str) -> Collection:
        try:
            return self.db[collection_name]
        except:
            return None

    async def insert_job_async(self, collection_name: str, **kwargs) -> Dict[str, Any]:
        return self.insert_job(collection_name, **kwargs)

    def insert_job(self, collection_name: str, **kwargs) -> Dict[str, Any]:
        coll = self.get_collection(collection_name)
        job_doc = kwargs.copy()
        print("Inserting job...")
        coll.insert_one(job_doc)
        print(f"Job successfully inserted: {self.db.pending_jobs.find_one(kwargs)}.")
        return kwargs

    async def update_job_status(self, collection_name: str, job_id: str, status: str | JobStatus):
        job_status = self._parse_enum_input(status)
        return self.db[collection_name].update_one({'job_id': job_id, }, {'$set': {'status': job_status}})

    def _parse_enum_input(self, _input: Any) -> str:
        return _input.value if isinstance(_input, Enum) else _input


In [48]:
uri = 'mongodb://localhost:27017/?retryWrites=true&w=majority&appName=biosimulators-processes'
conn = MongoDbConnector(connection_uri=uri, database_id="processes")

conn.refresh_data()



In [49]:
conn.data

{'process_results': []}

In [106]:
# run composite with copasi process and ram emitter
# on each update: return emitter val
# use db connector to write emitter val for update
# repeat


def run_composite_sse(duration, model_fp=model_fp, core=CORE):
    from process_bigraph import Composite 
    from uuid import uuid4

    uri = 'mongodb://localhost:27017/?retryWrites=true&w=majority&appName=biosimulators-processes'
    conn = MongoDbConnector(connection_uri=uri, database_id="processes")
    doc = {
        'copasi': {
            '_type': 'process',
            'address': 'local:copasi-process',
            'config': {
                'model': {
                    'model_source': model_fp
                }
            },
            'inputs': {
                'time': ['time_store'],
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'reactions': ['reactions_store']
            },
            'outputs': {
                'time': ['time_store'],
                'floating_species_concentrations': ['floating_species_concentrations_store'],
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'time': 'float',
                    'floating_species_concentrations': 'tree[float]'
                }
            },
            'inputs': {
                'time': ['time_store'],
                'floating_species_concentrations': ['floating_species_concentrations_store']
            }
        }
    }
    
    # make new job 
    job_id = str(uuid4())
    new_job = {'job_id': job_id}
    
    # insert mutable params
    write_job = new_job.copy()
    write_job.update({'last_updated': conn.timestamp(), 'data': []})
    job = conn.write(collection_name="process_results", **write_job)
    
    # make composite
    composite = Composite(config={'state': doc}, core=core)
    
    for n in range(duration):
        # run composite
        composite.run(1)
        
        # get historical results    
        results = composite.gather_results()
        data = results[('emitter',)]
        
        # find job and update data
        write_data = conn.db.process_results.find_one(new_job)
        write_data['data'] = data 
        
        # update db
        conn.db.process_results.update_one(new_job, {'$set': write_data})
        
    return conn

conn.refresh_data()
conn = run_composite_sse(10)

found a filepath


In [107]:
conn.data.get('process_results')

[{'_id': ObjectId('66df318e3c05234ee7df709b'),
  'job_id': 'e7619d85-c0a6-40df-9f30-e43e1a0a92fb',
  'last_updated': '2024-09-09 17:34:06.250744',
  'data': [{'time': 0.0,
    'floating_species_concentrations': {'LacI protein': 0.0,
     'TetR protein': 0.0,
     'cI protein': 0.0,
     'LacI mRNA': 0.0,
     'TetR mRNA': 20.0,
     'cI mRNA': 0.0}},
   {'time': 1.0,
    'floating_species_concentrations': {'LacI protein': 81.44046884923394,
     'TetR protein': 188.3821831030562,
     'cI protein': 42.64131125320374,
     'LacI mRNA': 19.90344166947679,
     'TetR mRNA': 50.61558010446934,
     'cI mRNA': 7.491001903157758}},
   {'time': 2.0,
    'floating_species_concentrations': {'LacI protein': 299.9792524293013,
     'TetR protein': 546.4092678039393,
     'cI protein': 127.23014070279017,
     'LacI mRNA': 41.13697667160224,
     'TetR mRNA': 74.22450737953378,
     'cI mRNA': 13.357268321541767}},
   {'time': 3.0,
    'floating_species_concentrations': {'LacI protein': 637.601781

In [60]:
x = [{'a': list(range(10))}, {'a': list(range(10))}]

list(set(x))

TypeError: unhashable type: 'dict'

In [103]:
s = {'a': 10}

s.update({'b': 3})

In [104]:
s

{'a': 10, 'b': 3}